In [ ]:

library(caret)

titanic = read.csv("C:/Users/student/Desktop/Senior/MATH 421 - R/assignment/train.csv")
#correct data type
titanic$Survived = factor(titanic$Survived)
titanic$Pclass = factor(titanic$Pclass)
titanic$Name = NULL
titanic$PassengerId = NULL
titanic$Ticket = NULL
titanic$Cabin = NULL
titanic$NumPeople = titanic$SibSp + titanic$Parch  #combine sibsp and parch as NumPeople
titanic$SibSp = NULL
titanic$Parch = NULL

handlemiss = function(data)
{
  for (i in 1:ncol(data)){
    if (class(data[, i]) != "factor") 
    { 
      data[,i][is.na(data[,i])]= mean(data[,i], na.rm = TRUE)   
    } else {
      levels=unique(data[,i])
      data[,i][is.na(data[,i])]=levels[which.max(tabulate(match(data[,i], levels)))]
    } 
  }
  return(data)
}
titanic = handlemiss(titanic)


adult = read.csv("C:/Users/student/Desktop/Senior/MATH 421 - R/assignment/adult data.csv", header = FALSE)
names(adult) = c("age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country", "target")
groupcat = adult
levels(groupcat$education) = c(replicate(3, "Some High School"), replicate(4, "< High School"), "Some College", "Some College", "College", "Grad or above", "Some College", "Grad or above", "< High School", "Grad or above", "Some College")
levels(groupcat$occupation) = c("Office", "Army", "Labor", "Manager", "Others", "Labor", "Labor", "Labor", "Labor","Labor", "Manager", "Army", "Office","Office", "Labor")  
levels(groupcat$`native_country`) = c("I", "A", "N", "A", "S", "I", "I", "S", "I", "E", "E", "E", "E", "N", "I", "E", "N", "A", "E", "A","A", "E", "E", "N", "A", "A", "N", "N", "I", "S","A", "E", "E", "N", "E","I", "A", "A", "I", "N", "A", "E")
adult = groupcat


prepro1 = preProcess(titanic, method = 'medianImpute')
titanic = predict(prepro1, newdata = titanic)

prepro2 = preProcess(adult, method = 'medianImpute')
audult = predict(prepro2, newdata = adult)


myGrid = expand.grid(mtry = 2, splitrule = c("gini"), min.node.size = c(1:3))
model_A =  train(target ~.,data = adult, method = "ranger", trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),tuneGrid = myGrid)
pred_A = predict(model_A, adult)
cm_A=confusionMatrix(data = pred_A, reference = adult$target, positive = " >50K")
print(cm_A)


model_T =  train(Survived ~.,data = titanic, method = "ranger", trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),tuneGrid = myGrid)
pred_T = predict(model_T, titanic)
cm_T = confusionMatrix(data = pred_T, reference = titanic$Survived, positive = "1")
print(cm_T)


model_A2 =train(target~.,data =adult, method = "rpart", trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))
pred_A2 = predict(model_A2,adult)
cm_A2 =confusionMatrix(pred_A2, adult$target, positive=" >50K")
print(cm_A2)


model_T2 = train(Survived~.,data =titanic, method = "rpart", trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))
pred_T2 = predict(model_T2,titanic)
cm_T2 =confusionMatrix(pred_T2, titanic$Survived, positive="1")
print(cm_T2)


model_A
model_T

library(rpart)
plot(model_A)
plot(model_T)


myGrid = expand.grid(mtry = 2, splitrule = c("gini", "extratrees"), min.node.size = c(1:4))
model_A3 =  train(target ~.,data = adult, method = "ranger", trControl = trainControl(method ="cv", number = 3, verboseIter = TRUE),tuneGrid = myGrid)
model_A3


myGrid = expand.grid(mtry = 2, splitrule = c("gini",  "extratrees"), min.node.size = c(1:4))
model_T3 =  train(Survived ~.,data = titanic, method = "ranger", trControl = trainControl(method ="cv", number = 3, verboseIter = TRUE),tuneGrid = myGrid)

plot(model_A3)
plot(model_T3)

library(glmnet)
myControl <- trainControl(method = "cv", number = 5, verboseIter = TRUE)
model_T4 <- train(Survived ~. ,tuneLength = 1,  titanic, method = "glmnet", trControl = myControl)
pred_T4 = predict(model_T4,titanic)
cm_T4 =confusionMatrix(pred_T4, titanic$Survived, positive="1")
print(cm_T4)

model_T5 <- train(Survived ~. ,tuneLength = 1,  titanic, tuneGrid = expand.grid(alpha = 0:1, lambda = seq(0.0001, 0.1, length = 20)),method = "glmnet", trControl = myControl)
pred_T5 = predict(model_T5,titanic)
cm_T5 =confusionMatrix(pred_T5, titanic$Survived, positive="1")
print(cm_T5)

model_A4 <- train(target ~. ,tuneLength = 1,  adult, method = "glmnet", trControl = myControl)
pred_A4 = predict(model_A4,adult)
cm_A4 =confusionMatrix(pred_A4, adult$target, positive=" >50K")
print(cm_A4)

model_A5 <- train(target~. , tuneLength = 1, adult, tuneGrid = expand.grid(alpha = 0:1, lambda = seq(0.0001, 0.01, length = 20)),method = "glmnet", trControl = myControl)
pred_A5 = predict(model_A5,adult)
cm_A5 =confusionMatrix(pred_A5, adult$target, positive=" >50K")
print(cm_A5)

